In [1]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import earthaccess
import h5netcdf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyinterp.backends.xarray  # Module that handles the filling of undefined values.
import pyinterp.fill
import seaborn as sns
import xarray as xr
from matplotlib.patches import Rectangle
import matplotlib.patches as mpatches
sns.set_style("whitegrid")
sns.set_context("talk", font_scale=1.2)  

# Access MOANA rasters

In [2]:
auth = earthaccess.login()

In [3]:
tspan = ("2024-04-01", "2025-08-31")

In [4]:
results_moana = earthaccess.search_data(
    short_name="PACE_OCI_L3M_MOANA",
    granule_name="*.Day.*0p1deg*",  # Daily: Day | Resolution: 0p1deg or 4 (for 4km)
    temporal=tspan,
)

In [5]:
def time_from_attr(ds):
    """Set the time attribute as a dataset variable
    Args:
        ds: a dataset corresponding to one or multiple Level-2 granules
    Returns:
        the dataset with a scalar "time" coordinate
    """
    datetime = ds.attrs["time_coverage_start"].replace("Z", "")
    ds["date"] = ((), np.datetime64(datetime, "ns"))
    ds = ds.set_coords("date")
    return ds

In [6]:
path_files = earthaccess.open(results_moana)

QUEUEING TASKS | :   0%|          | 0/376 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/376 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/376 [00:00<?, ?it/s]

In [7]:
dataset_moana = xr.open_mfdataset(
    path_files, preprocess=time_from_attr, combine="nested", concat_dim="date"
)
dataset_moana

<xarray.Dataset> Size: 7GB
Dimensions:          (date: 376, lat: 1400, lon: 1100, rgb: 3,
                      eightbitcolor: 256)
Coordinates:
  * lat              (lat) float32 6kB 69.95 69.85 69.75 ... -69.85 -69.95
  * lon              (lon) float32 4kB -84.95 -84.85 -84.75 ... 24.85 24.95
  * date             (date) datetime64[ns] 3kB 2024-04-01T11:01:27 ... 2025-0...
Dimensions without coordinates: rgb, eightbitcolor
Data variables:
    prococcus_moana  (date, lat, lon) float32 2GB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    syncoccus_moana  (date, lat, lon) float32 2GB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    picoeuk_moana    (date, lat, lon) float32 2GB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    palette          (date, rgb, eightbitcolor) uint8 289kB dask.array<chunksize=(1, 3, 256), meta=np.ndarray>
Attributes: (12/62)
    product_name:                      PACE_OCI.20240401.L3m.DAY.MOANA.V3_0.0...
    instrument:                        OCI
    title:                             OCI Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          PACE
    source:                            satellite observations from OCI-PACE
    ...                                ...
    cdm_data_type:                     grid
    identifier_product_doi_authority:  http://dx.doi.org
    identifier_product_doi:            10.5067/PACE/OCI/L3M/MOANA/3.0
    data_bins:                         166600
    data_minimum:                      -inf
    data_maximum:                      inf

## Clean data

In [8]:
dataset_moana["prococcus_moana"] = dataset_moana["prococcus_moana"].clip(
    min=dataset_moana["prococcus_moana"].attrs["valid_min"],
    max=dataset_moana["prococcus_moana"].attrs["valid_max"],
)
dataset_moana["syncoccus_moana"] = dataset_moana["syncoccus_moana"].clip(
    min=dataset_moana["syncoccus_moana"].attrs["valid_min"],
    max=dataset_moana["syncoccus_moana"].attrs["valid_max"],
)
dataset_moana["picoeuk_moana"] = dataset_moana["picoeuk_moana"].clip(
    min=dataset_moana["picoeuk_moana"].attrs["valid_min"],
    max=dataset_moana["picoeuk_moana"].attrs["valid_max"],
)

In [9]:
dataset_phy = dataset_moana.drop_vars(["palette"])

In [30]:
moana_pcc_monthly_mean = dataset_phy.resample(date="1M").mean("date")

# Access PACE BGC rasters

## PACE POC

In [10]:
results_pace_poc = earthaccess.search_data(
    short_name="PACE_OCI_L3M_POC",
    granule_name="*.Day.*0p1deg*",  # Daily: Day | Resolution: 0p1deg or 4 (for 4km)
    temporal=tspan,
)

In [11]:
path_files = earthaccess.open(results_pace_poc)

QUEUEING TASKS | :   0%|          | 0/387 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/387 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/387 [00:00<?, ?it/s]

In [12]:
dataset_pace_poc = xr.open_mfdataset(
    path_files, preprocess=time_from_attr, combine="nested", concat_dim="date"
)
dataset_pace_poc

<xarray.Dataset> Size: 10GB
Dimensions:  (date: 387, lat: 1800, lon: 3600, rgb: 3, eightbitcolor: 256)
Coordinates:
  * lat      (lat) float32 7kB 89.95 89.85 89.75 89.65 ... -89.75 -89.85 -89.95
  * lon      (lon) float32 14kB -179.9 -179.9 -179.8 ... 179.8 179.9 180.0
  * date     (date) datetime64[ns] 3kB 2024-03-31T23:33:05 ... 2025-05-01T23:...
Dimensions without coordinates: rgb, eightbitcolor
Data variables:
    poc      (date, lat, lon) float32 10GB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    palette  (date, rgb, eightbitcolor) uint8 297kB dask.array<chunksize=(1, 3, 256), meta=np.ndarray>
Attributes: (12/64)
    product_name:                      PACE_OCI.20240401.L3m.DAY.POC.V3_0.poc...
    instrument:                        OCI
    title:                             OCI Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          PACE
    source:                            satellite observations from OCI-PACE
    ...                                ...
    identifier_product_doi:            10.5067/PACE/OCI/L3M/POC/3.0
    keywords:                          Earth Science > Oceans > Ocean Chemist...
    keywords_vocabulary:               NASA Global Change Master Directory (G...
    data_bins:                         554519
    data_minimum:                      9.0
    data_maximum:                      1953.0

### Clean data

In [14]:
dataset_pace_poc["poc"] = dataset_pace_poc["poc"].clip(
    min=dataset_pace_poc["poc"].attrs["valid_min"],
    max=dataset_pace_poc["poc"].attrs["valid_max"],
)

dataset_pace_poc = dataset_pace_poc.drop_vars(["palette"])

In [16]:
pace_poc_monthly_mean = dataset_pace_poc.resample(date="1M").mean("date")

## PACE Carbon

In [17]:
results_pace_car = earthaccess.search_data(
    short_name="PACE_OCI_L3M_CARBON",
    granule_name="*.Day.*0p1deg*",  # Daily: Day | Resolution: 0p1deg or 4 (for 4km)
    temporal=tspan,
)

In [18]:
path_files = earthaccess.open(results_pace_car)

QUEUEING TASKS | :   0%|          | 0/387 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/387 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/387 [00:00<?, ?it/s]

In [21]:
dataset_pace_car = xr.open_mfdataset(
    path_files, preprocess=time_from_attr, combine="nested", concat_dim="date"
)
dataset_pace_car

<xarray.Dataset> Size: 10GB
Dimensions:       (date: 387, lat: 1800, lon: 3600, rgb: 3, eightbitcolor: 256)
Coordinates:
  * lat           (lat) float32 7kB 89.95 89.85 89.75 ... -89.75 -89.85 -89.95
  * lon           (lon) float32 14kB -179.9 -179.9 -179.8 ... 179.8 179.9 180.0
  * date          (date) datetime64[ns] 3kB 2024-03-31T23:33:05 ... 2025-05-0...
Dimensions without coordinates: rgb, eightbitcolor
Data variables:
    carbon_phyto  (date, lat, lon) float32 10GB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    palette       (date, rgb, eightbitcolor) uint8 297kB dask.array<chunksize=(1, 3, 256), meta=np.ndarray>
Attributes: (12/62)
    product_name:                      PACE_OCI.20240401.L3m.DAY.CARBON.V3_0....
    instrument:                        OCI
    title:                             OCI Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          PACE
    source:                            satellite observations from OCI-PACE
    ...                                ...
    cdm_data_type:                     grid
    identifier_product_doi_authority:  http://dx.doi.org
    identifier_product_doi:            10.5067/PACE/OCI/L3M/CARBON/3.0
    data_bins:                         554626
    data_minimum:                      0.59064007
    data_maximum:                      1000.0002

### Clean data

In [23]:
dataset_pace_car["carbon_phyto"] = dataset_pace_car["carbon_phyto"].clip(
    min=dataset_pace_car["carbon_phyto"].attrs["valid_min"],
    max=dataset_pace_car["carbon_phyto"].attrs["valid_max"],
)

dataset_pace_car = dataset_pace_car.drop_vars(["palette"])

In [24]:
pace_car_monthly_mean = dataset_pace_car.resample(date="1M").mean("date")

## PACE Chl

In [25]:
results_pace_chl = earthaccess.search_data(
    short_name="PACE_OCI_L3M_CHL",
    granule_name="*.Day.*0p1deg*",  # Daily: Day | Resolution: 0p1deg or 4 (for 4km)
    temporal=tspan,
)

In [26]:
path_files = earthaccess.open(results_pace_chl)

QUEUEING TASKS | :   0%|          | 0/387 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/387 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/387 [00:00<?, ?it/s]

In [27]:
dataset_pace_chl = xr.open_mfdataset(
    path_files, preprocess=time_from_attr, combine="nested", concat_dim="date"
)
dataset_pace_chl

<xarray.Dataset> Size: 10GB
Dimensions:  (date: 387, lat: 1800, lon: 3600, rgb: 3, eightbitcolor: 256)
Coordinates:
  * lat      (lat) float32 7kB 89.95 89.85 89.75 89.65 ... -89.75 -89.85 -89.95
  * lon      (lon) float32 14kB -179.9 -179.9 -179.8 ... 179.8 179.9 180.0
  * date     (date) datetime64[ns] 3kB 2024-03-31T23:33:05 ... 2025-05-01T23:...
Dimensions without coordinates: rgb, eightbitcolor
Data variables:
    chlor_a  (date, lat, lon) float32 10GB dask.array<chunksize=(1, 512, 1024), meta=np.ndarray>
    palette  (date, rgb, eightbitcolor) uint8 297kB dask.array<chunksize=(1, 3, 256), meta=np.ndarray>
Attributes: (12/64)
    product_name:                      PACE_OCI.20240401.L3m.DAY.CHL.V3_0.chl...
    instrument:                        OCI
    title:                             OCI Level-3 Standard Mapped Image
    project:                           Ocean Biology Processing Group (NASA/G...
    platform:                          PACE
    source:                            satellite observations from OCI-PACE
    ...                                ...
    identifier_product_doi:            10.5067/PACE/OCI/L3M/CHL/3.0
    keywords:                          Earth Science > Oceans > Ocean Chemist...
    keywords_vocabulary:               NASA Global Change Master Directory (G...
    data_bins:                         555432
    data_minimum:                      0.005920372
    data_maximum:                      90.442894

### Clean data

In [28]:
dataset_pace_chl["chlor_a"] = dataset_pace_chl["chlor_a"].clip(
    min=dataset_pace_chl["chlor_a"].attrs["valid_min"],
    max=dataset_pace_chl["chlor_a"].attrs["valid_max"],
)

dataset_pace_chl = dataset_pace_chl.drop_vars(["palette"])

In [29]:
pace_chl_monthly_mean = dataset_pace_chl.resample(date="1M").mean("date")

# PC1 Map